In [1]:
import tensorflow as tf
import qresnet
import dataset
from tensorflow import keras
from keras import layers
from qkeras import *

In [2]:
x_train, y_train, x_test, y_test = dataset.load_cifar100()

x_train /= 255.0
x_test /= 255.0


In [3]:
x = x_in = layers.Input(x_train.shape[1:], name="input")

x = qresnet.resnet18(x)

qmodel = keras.Model(inputs=[x_in], outputs=[x])
qmodel.summary()


c:\Users\georg\Desktop\mestrado_luiz\deep_nibble_dnn\.venv\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer HeNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 34, 34, 3)    0           ['input[0][0]']                  
                                                                                                  
 qconv1 (Conv2D)                (None, 32, 32, 16)   432         ['conv1_pad[0][0]']              
                                                                                                  
 bn1 (BatchNormalization)       (None, 32, 32, 16)   64          ['qconv1[0][0]']                 
                                                                                              

In [4]:
class ResNetPaperLR(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_lr = 0.1, lr_decay=10, steps=[32000, 48000]):
        super(ResNetPaperLR, self).__init__()
        self.initial_lr = initial_lr
        self.learning_rate_decay = lr_decay
        self.steps = steps
        self.current_lr = 0
    def __call__(self, step):        
        step12 = tf.where(step < self.steps[1], self.initial_lr/self.learning_rate_decay, self.initial_lr/(self.learning_rate_decay**2))    
        step01 = tf.where(step < self.steps[0], self.initial_lr, step12)
        return step01
                         

    def get_config(self):
        return {
            "initial_lr": self.initial_lr,
            "learning_rate_decay": self.learning_rate_decay,
            "steps": self.steps
        }


NB_EPOCH = 164
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10
OPTIMIZER = keras.optimizers.SGD(learning_rate=ResNetPaperLR(), momentum=0.9)


qmodel.compile(optimizer=OPTIMIZER, loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.02), metrics=["accuracy"])


In [5]:
history = qmodel.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, initial_epoch=0, verbose=VERBOSE, validation_data=(x_test, y_test), validation_freq=2)


Epoch 1/164
391/391 [==============================] - 30s 62ms/step - loss: 4.2857 - accuracy: 0.0477
Epoch 2/164
391/391 [==============================] - 29s 74ms/step - loss: 3.9569 - accuracy: 0.0871 - val_loss: 4.0014 - val_accuracy: 0.0840
Epoch 3/164
391/391 [==============================] - 24s 62ms/step - loss: 3.7773 - accuracy: 0.1208
Epoch 4/164
391/391 [==============================] - 27s 70ms/step - loss: 3.6178 - accuracy: 0.1500 - val_loss: 3.6152 - val_accuracy: 0.1510
Epoch 5/164
391/391 [==============================] - 24s 62ms/step - loss: 3.4829 - accuracy: 0.1737
Epoch 6/164
391/391 [==============================] - 28s 71ms/step - loss: 3.3645 - accuracy: 0.1941 - val_loss: 3.6665 - val_accuracy: 0.1482
Epoch 7/164
391/391 [==============================] - 25s 63ms/step - loss: 3.2644 - accuracy: 0.2116
Epoch 8/164
391/391 [==============================] - 28s 71ms/step - loss: 3.1808 - accuracy: 0.2307 - val_loss: 3.4292 - val_accuracy: 0.1957
Epoch 9/